# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# # Import API key

from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Loading in WeatherPY CSV

input_file="../output_data/cities.csv"


csv=pd.read_csv(input_file)

csv.head()

,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tūkrah,32.53,20.58,63.21,83,4,12.73,LY,1587672213
1,1,Dikson,73.51,80.55,35.76,84,100,24.85,RU,1587672214
2,2,Chui,-33.70,-53.46,66.25,88,14,11.48,UY,1587672214
3,3,Jamestown,42.10,-79.24,53.60,54,75,3.36,US,1587672214
4,4,Vaini,-21.20,-175.20,77.00,94,75,5.82,TO,1587672214


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Creating necesarry variables for Location and Humidity
locations=csv[["Lat", "Lng"]]
Humidity= csv["Humidity"]

#Define the heatmap parameters
fig=gmaps.figure()

#Create heat layer
heat_layer=gmaps.heatmap_layer(locations, weights=Humidity, dissipating= False, max_intensity=250,
                              point_radius=3)

#Add Layer
fig.add_layer(heat_layer)

#Display the map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Personal preference
    #80> but not higher 90 
    #50< Humidity
    #Lat > 20

temp_df=csv[(csv["Max Temp"]>=80)&(csv["Max Temp"]<90)&(csv["Humidity"]<50)& (csv["Lat"]>20)]
temp_df

,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,Jalpa,21.63,-102.97,84.20,5,75,2.24,MX,1587672215
33,33,Cabo San Lucas,22.89,-109.91,86.00,28,5,4.70,MX,1587672000
61,61,Tessalit,20.20,1.01,88.20,9,0,11.92,ML,1587672127
130,130,Guerrero Negro,27.98,-114.06,82.13,32,1,14.25,MX,1587672236
148,148,Riyadh,24.69,46.72,80.60,39,0,3.96,SA,1587672012
204,204,Actopan,20.27,-98.93,83.59,12,81,5.48,MX,1587672249
268,268,Laguna,38.42,-121.42,82.00,32,1,10.29,US,1587672260
282,282,Atar,20.52,-13.05,81.61,14,0,8.59,MR,1587672262
370,370,Lufkin,31.34,-94.73,82.00,44,0,13.87,US,1587672278
474,474,León,21.12,-101.67,86.00,12,75,5.82,MX,1587672158


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#create df of hotel name, along with city, count, geocod(lat,long)
hotel_df=temp_df[["City","Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Names"]= ""
hotel_df

,City,Country,Lat,Lng,Hotel Names
8,Jalpa,MX,21.63,-102.97,
33,Cabo San Lucas,MX,22.89,-109.91,
61,Tessalit,ML,20.20,1.01,
130,Guerrero Negro,MX,27.98,-114.06,
148,Riyadh,SA,24.69,46.72,
204,Actopan,MX,20.27,-98.93,
268,Laguna,US,38.42,-121.42,
282,Atar,MR,20.52,-13.05,
370,Lufkin,US,31.34,-94.73,
474,León,MX,21.12,-101.67,


In [6]:
#Define variables
radius=8000
gtype="lodging"

#Making query calls for top Hotel

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params={"radius": radius, "types": gtype,"key": g_key}



#Using .itterrow()

for index, row in hotel_df.iterrows():
    
    #get city name in each row
    location=row[["Lat","Lng"]].values

    #add a keyword to my params dictionary
    params["location"]=f'{location[0]},{location[1]}'

    #Make a query url and API call
    query_url=requests.get(base_url, params=params).json()
    
    #extract results
    results=query_url['results']
    
    #Add the information into the Hotel Names column
    try:
        hotel_df.loc[index, "Hotel Names"]=results[0]['name']
    except (KeyError, IndexError):
        print("There is a missing field. Skipping...")

There is a missing field. Skipping...


In [7]:
hotel_df=hotel_df[hotel_df["Hotel Names"]!= ""]

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Names}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)


# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))